### lightGBM実行用ファイル

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib

###訓練データの読み込み
house_age = pd.read_csv('house_age.csv')
area_size = pd.read_csv('area_size.csv')
room_arrange = pd.read_csv('room_arrange.csv')
contract_span = pd.read_csv('contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('reg_rent.csv')
floor_scores = pd.read_csv('floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('stations.csv')
minits = pd.read_csv('minits.csv')
addresses = pd.read_csv('addresses.csv')
room_arrange = pd.read_csv('room_arrange.csv')
buildings = pd.read_csv('buildings.csv')

rent = pd.read_csv('rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('test_house_age.csv')
test_area_size = pd.read_csv('test_area_size.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_contract_span = pd.read_csv('test_contract_span.csv')
test_contract_span2 = pd.read_csv('test_contract_span2.csv')
test_reg_rent = pd.read_csv('test_reg_rent.csv')
test_floor_scores = pd.read_csv('test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('test_capital_floor_scores2.csv')
test_stations = pd.read_csv('test_stations.csv')
test_minits = pd.read_csv('test_minits.csv')
test_addresses = pd.read_csv('test_addresses.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_buildings = pd.read_csv('test_buildings.csv')

契約期間２＆フル間取りver.(チューニング済み)

In [46]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K","S"]]], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D","K","S"]]], axis=1)

category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','K','S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'reg_lambda': 4.430375245218262e-06,
    'max_bin': 502,
    'num_leaves': 97
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
display(importance)


c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a futur

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003017 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2341
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 14
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 35409.7	valid_1's rmse: 41017.7
[20]	training's rmse: 20985.8	valid_1's rmse: 29980.8
[30]	training's rmse: 15796.7	valid_1's rmse: 26085.4
[40]	training's rmse: 13390.6	valid_1's rmse: 24508
[50]	training's rmse: 12056	valid_1's rmse: 23796.5
[60]	training's rmse: 11139.5	valid_1's rmse: 23322.4
[70]	training's rmse: 10400.3	valid_1's rmse: 22892
[80]	training's rmse: 9664.53	valid_1's rmse: 22571.6
[90]	training's rmse: 9051.47	valid_1's rmse: 22353.7
[100]	training's rmse: 8593.83	valid_1's rmse: 22200.4
[110]	training's rmse: 8186.38	valid_1's rmse: 22010
[

,importance
築年数,4.008499e+13
面積,2.431580e+14
契約期間,6.582397e+12
定期借家,1.857613e+12
所在階,4.746837e+12
全体の階数,3.579679e+13
最寄り駅,7.055608e+13
所要時間,8.187477e+12
所在地,2.802448e+13
部屋数,2.472790e+12


In [47]:
df = pd.read_csv('test.csv')
id = df['id']
y_pred = pd.DataFrame(y_pred)
result = pd.concat([id, y_pred],axis=1)
result.to_csv('LGBM_cs2_nLDKS_result.csv',index=False)

契約期間１＆フル間取りのチューニング

In [48]:
X_train = pd.concat([house_age, area_size, contract_span, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

def objective(trial):
    category_lists = ['定期借家', '最寄り駅', '所在地', 'L', 'D', 'K', 'S']
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

    params = {
        'objective':'regression',
        'metrics':'rmse',
        'learning_rate':0.05,
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0000001, 0.0001, log=True),
        'max_bin': trial.suggest_int('max_bin', 255, 600),
        'num_leaves': trial.suggest_int('num_leaves', 50, 128),
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        #verbose_eval=10, 
                        num_boost_round=3000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)

    loss = mean_squared_error(y_valid, y_pred_valid, squared=False)
    return loss

In [49]:
# study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
# study.optimize(objective, n_trials=40)
# study.best_params

契約期間１＆フル間取り（チューニング済）ver.

In [50]:
X_train = pd.concat([house_age, area_size, contract_span, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K","S"]]], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D","K","S"]]], axis=1)

category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','K','S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'reg_lambda': 4.430375245218262e-06,
    'max_bin': 502,
    'num_leaves': 97
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
display(importance)


c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a futur

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003449 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2341
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 14
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 35676.3	valid_1's rmse: 41313.5
[20]	training's rmse: 21063.6	valid_1's rmse: 30049.6
[30]	training's rmse: 15778.4	valid_1's rmse: 26238.9
[40]	training's rmse: 13423.6	valid_1's rmse: 24739.4
[50]	training's rmse: 12032.4	valid_1's rmse: 23979.9
[60]	training's rmse: 11076.2	valid_1's rmse: 23508.7
[70]	training's rmse: 10221.3	valid_1's rmse: 23063.7
[80]	training's rmse: 9566.95	valid_1's rmse: 22775.7
[90]	training's rmse: 8999.34	valid_1's rmse: 22582
[100]	training's rmse: 8514.84	valid_1's rmse: 22377.3
[110]	training's rmse: 8135.44	valid_1's rmse: 222

,importance
築年数,4.008042e+13
面積,2.431145e+14
契約期間,3.500580e+12
定期借家,3.284547e+12
所在階,4.699028e+12
全体の階数,3.603359e+13
最寄り駅,7.036812e+13
所要時間,8.324592e+12
所在地,2.791183e+13
部屋数,3.116196e+12


契約期間１＆フル間取り（チューニング済み）ver.のファイル出力

In [51]:
df = pd.read_csv('test.csv')
id = df['id']
y_pred = pd.DataFrame(y_pred)
result = pd.concat([id, y_pred],axis=1)
result.to_csv('LGBM_cs1_nLDKS_result.csv',index=False)

契約期間１と２のバージョン差による相関関係が高すぎるので、契約期間２だけをアンサンブルで用いる。

In [52]:
LGBM_cal1_nLDKS_results = pd.read_csv('LGBM_cs1_nLDKS_result.csv')
LGBM_cal2_nLDKS_results = pd.read_csv('LGBM_cs2_nLDKS_result.csv')
X = pd.concat([LGBM_cal1_nLDKS_results['0'],LGBM_cal2_nLDKS_results['0']],axis=1)
X.corr()

,0,0
0,1.000000,0.998007
0,0.998007,1.000000


契約期間２＆間取り（S以外）のチューニング

In [53]:
# X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K"]]], axis=1)
# y_train = rent

# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

# X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D","K"]]], axis=1)

# def objective(trial):
#     category_lists = ['定期借家', '最寄り駅', '所在地', 'L', 'D', 'K']
#     lgb_train = lgb.Dataset(X_train, y_train)
#     lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

#     params = {
#         'objective':'regression',
#         'metrics':'rmse',
#         'learning_rate':0.05,
#         'reg_lambda': trial.suggest_float('reg_lambda', 0.0000001, 0.0001, log=True),
#         'max_bin': trial.suggest_int('max_bin', 255, 600),
#         'num_leaves': trial.suggest_int('num_leaves', 50, 128),
#     }

#     model = lgb.train(
#                         params,
#                         lgb_train, 
#                         valid_sets=[lgb_train, lgb_eval], 
#                         #verbose_eval=10, 
#                         num_boost_round=3000, 
#                         early_stopping_rounds=10,
#                         categorical_feature = category_lists
#                         )

#     y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)

#     loss = mean_squared_error(y_valid, y_pred_valid, squared=False)
#     return loss

In [54]:
# study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
# study.optimize(objective, n_trials=40)
# study.best_params

契約期間２＆間取り（S以外）

In [55]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K"]]], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D","K"]]], axis=1)

category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','K']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'reg_lambda': 4.430375245218262e-06,
    'max_bin': 502,
    'num_leaves': 97
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
display(importance)


c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future rel

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003105 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2338
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 35409.7	valid_1's rmse: 41017.7
[20]	training's rmse: 20992.8	valid_1's rmse: 29987.9
[30]	training's rmse: 15816.3	valid_1's rmse: 26143
[40]	training's rmse: 13448.1	valid_1's rmse: 24530.4
[50]	training's rmse: 12097.3	valid_1's rmse: 23854.2
[60]	training's rmse: 11187.3	valid_1's rmse: 23345.5
[70]	training's rmse: 10408.8	valid_1's rmse: 22980.3
[80]	training's rmse: 9712	valid_1's rmse: 22660.9
[90]	training's rmse: 9103.33	valid_1's rmse: 22366.5
[100]	training's rmse: 8602.63	valid_1's rmse: 22142.6
[110]	training's rmse: 8220.1	valid_1's rmse: 21988.4

,importance
築年数,4.031494e+13
面積,2.434538e+14
契約期間,6.287400e+12
定期借家,1.866982e+12
所在階,4.686737e+12
全体の階数,3.580639e+13
最寄り駅,7.062095e+13
所要時間,8.290599e+12
所在地,2.796149e+13
部屋数,2.570306e+12


In [56]:
df = pd.read_csv('test.csv')
id = df['id']
y_pred = pd.DataFrame(y_pred)
result = pd.concat([id, y_pred],axis=1)
result.to_csv('LGBM_cs2_nLDK_result.csv',index=False)

間取りのSがあるかないかの相関関係も高すぎるので、アンサンブルには契約期間２＆間取り（S以外）を採用する。

In [57]:
LGBM_cal2_nLDKS_results = pd.read_csv('LGBM_cs2_nLDKS_result.csv')
LGBM_cal2_nLDK_results = pd.read_csv('LGBM_cs2_nLDK_result.csv')
X = pd.concat([LGBM_cal2_nLDKS_results['0'],LGBM_cal2_nLDK_results['0']],axis=1)
X.corr()

,0,0
0,1.000000,0.998769
0,0.998769,1.000000


lightGBMで契約期間１＆フル間取り＆建物構造

In [58]:
X_train = pd.concat([house_age, area_size, contract_span, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K","S"]], buildings], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D","K","S"]],test_buildings], axis=1)

category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','K','S','建物構造']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'reg_lambda': 4.430375245218262e-06,
    'max_bin': 502,
    'num_leaves': 97
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
display(importance)


c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '建物構造', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003991 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2344
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 15
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 35859.2	valid_1's rmse: 41406.3
[20]	training's rmse: 21186.4	valid_1's rmse: 30202.6
[30]	training's rmse: 15913.6	valid_1's rmse: 26355.3
[40]	training's rmse: 13548.7	valid_1's rmse: 24792.8
[50]	training's rmse: 12171.9	valid_1's rmse: 24039.4
[60]	training's rmse: 11071.3	valid_1's rmse: 23547.7
[70]	training's rmse: 10270.7	valid_1's rmse: 23205.2
[80]	training's rmse: 9491.94	valid_1's rmse: 22819.7
[90]	training's rmse: 8897.87	valid_1's rmse: 22589.1
[100]	training's rmse: 8406.53	valid_1's rmse: 22350.7
[110]	training's rmse: 8018.03	valid_1's rmse: 2

,importance
築年数,4.001278e+13
面積,2.410613e+14
契約期間,3.003898e+12
定期借家,3.388476e+12
所在階,4.979998e+12
全体の階数,3.617138e+13
最寄り駅,6.887100e+13
所要時間,8.619826e+12
所在地,2.774219e+13
部屋数,3.559488e+12


In [59]:
df = pd.read_csv('test.csv')
id = df['id']
y_pred = pd.DataFrame(y_pred)
result = pd.concat([id, y_pred],axis=1)
result.to_csv('LGBM_cs_nLDKS_bld_result.csv',index=False)

建物構造を入れたらさらに相関係数が上がってしまった。

In [60]:
LGBM_cs2_nLDK_results = pd.read_csv('LGBM_cs2_nLDK_result.csv')
LGBM_cs1_nLDKS_bld_results = pd.read_csv('LGBM_cs_nLDKS_bld_result.csv')
X = pd.concat([LGBM_cs2_nLDK_results['0'],LGBM_cs1_nLDKS_bld_results['0']],axis=1)
X.corr()

,0,0
0,1.000000,0.997632
0,0.997632,1.000000


lightGBMで契約期間２＆間取り（KS以外）＆建物構造を試してみる。

In [61]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D"]], buildings], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D"]],test_buildings], axis=1)

category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','建物構造']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'reg_lambda': 4.430375245218262e-06,
    'max_bin': 502,
    'num_leaves': 97
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
display(importance)


c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', '定期借家', '建物構造', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001191 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2338
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 35522.8	valid_1's rmse: 41119
[20]	training's rmse: 20999.8	valid_1's rmse: 30029.8
[30]	training's rmse: 15766.6	valid_1's rmse: 26169.6
[40]	training's rmse: 13470.8	valid_1's rmse: 24666.9
[50]	training's rmse: 12167.3	valid_1's rmse: 23997.3
[60]	training's rmse: 11211.1	valid_1's rmse: 23438.5
[70]	training's rmse: 10402.8	valid_1's rmse: 22959.8
[80]	training's rmse: 9724.98	valid_1's rmse: 22708.4
[90]	training's rmse: 9124.37	valid_1's rmse: 22473.9
[100]	training's rmse: 8603.84	valid_1's 

,importance
築年数,3.957611e+13
面積,2.419552e+14
契約期間,6.641633e+12
定期借家,1.841904e+12
所在階,4.469821e+12
全体の階数,3.555017e+13
最寄り駅,7.145723e+13
所要時間,8.618902e+12
所在地,2.742508e+13
部屋数,2.697620e+12


In [62]:
df = pd.read_csv('test.csv')
id = df['id']
y_pred = pd.DataFrame(y_pred)
result = pd.concat([id, y_pred],axis=1)
result.to_csv('LGBM_cs2_nLD_bld_result.csv',index=False)

lightGBMを使う限りは小手先の特徴量入れ替えでは対して変化しない模様・・・

In [63]:
LGBM_cs_nLDKS_results = pd.read_csv('LGBM_cs_nLDKS_result.csv')
LGBM_cs2_nLD_bld_results = pd.read_csv('LGBM_cs2_nLD_bld_result.csv')
X = pd.concat([LGBM_cs_nLDKS_results['0'],LGBM_cs2_nLD_bld_results['0']],axis=1)
X.corr()

,0,0
0,1.000000,0.997794
0,0.997794,1.000000


lightGBMで契約期間２＆間取り（KS以外）＆建物構造のチューニング

In [64]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D"]],buildings], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D"]],test_buildings], axis=1)

def objective(trial):
    category_lists = ['定期借家', '最寄り駅', '所在地', 'L', 'D', '建物構造']
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

    params = {
        'objective':'regression',
        'metrics':'rmse',
        'learning_rate':0.05,
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0000001, 0.0001, log=True),
        'max_bin': trial.suggest_int('max_bin', 255, 600),
        'num_leaves': trial.suggest_int('num_leaves', 50, 128),
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        #verbose_eval=10, 
                        num_boost_round=3000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)

    loss = mean_squared_error(y_valid, y_pred_valid, squared=False)
    return loss

In [65]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)
study.best_params

[I 2022-09-09 13:05:13,258] A new study created in memory with name: no-name-30e52d5f-dbcb-4cab-bdc0-2c9ae765a011
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', '定期借家', '建物構造', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\Users\kos

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001953 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2338
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[1]	training's rmse: 70607.7	valid_1's rmse: 71336.7
Training until validation scores don't improve for 10 rounds
[2]	training's rmse: 67764.5	valid_1's rmse: 68774.7
[3]	training's rmse: 65075.5	valid_1's rmse: 66356.1
[4]	training's rmse: 62535.5	valid_1's rmse: 64098.3
[5]	training's rmse: 60117.4	valid_1's rmse: 61992.9
[6]	training's rmse: 57856.8	valid_1's rmse: 59974.6
[7]	training's rmse: 55700.4	valid_1's rmse: 58123.5
[8]	training's rmse: 53678	valid_1's rmse: 56338.6
[9]	training's rmse: 51763	valid_1's rmse: 54701.3
[10]	training's rmse: 49941.8	valid_1's rmse: 53144.

[I 2022-09-09 13:05:32,877] Trial 0 finished with value: 19869.439942355075 and parameters: {'reg_lambda': 4.430375245218262e-06, 'max_bin': 502, 'num_leaves': 97}. Best is trial 0 with value: 19869.439942355075.
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', '定期借家', '建物構造', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from 

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004754 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2108
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[1]	training's rmse: 70602.3	valid_1's rmse: 71335.9
Training until validation scores don't improve for 10 rounds
[2]	training's rmse: 67708.5	valid_1's rmse: 68695.3
[3]	training's rmse: 64979.6	valid_1's rmse: 66242.1
[4]	training's rmse: 62442.7	valid_1's rmse: 63971.8
[5]	training's rmse: 60056	valid_1's rmse: 61843.3
[6]	training's rmse: 57736.9	valid_1's rmse: 59784.6
[7]	training's rmse: 55602.5	valid_1's rmse: 57924.5
[8]	training's rmse: 53584.4	valid_1's rmse: 56186
[9]	training's rmse: 51633.3	valid_1's rmse: 54484.3
[10]	training's rmse: 49821	valid_1's rmse: 52901.9
[11]	training's rmse: 48068.5	valid_1's rmse: 51391.9
[12]	train

[I 2022-09-09 13:05:59,055] Trial 1 finished with value: 20698.376192896878 and parameters: {'reg_lambda': 4.311710058685494e-06, 'max_bin': 401, 'num_leaves': 101}. Best is trial 0 with value: 19869.439942355075.
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', '定期借家', '建物構造', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from

...

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001346 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2085
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[1]	training's rmse: 70550.1	valid_1's rmse: 71215.6
Training until validation scores don't improve for 10 rounds
[2]	training's rmse: 67648	valid_1's rmse: 68540.1
[3]	training's rmse: 64931.7	valid_1's rmse: 66072.7
[4]	training's rmse: 62352.5	valid_1's rmse: 63737.3
[5]	training's rmse: 59914	valid_1's rmse: 61573.4
[6]	training's rmse: 57626	valid_1's rmse: 59541.9
[7]	training's rmse: 55461.3	valid_1's rmse: 57606.6
[8]	training's rmse: 53433.7	valid_1's rmse: 55829.8
[9]	training's rmse: 51494.8	valid_1's rmse: 54137.2
[10]	training's rmse: 49661.1	valid_1's rmse: 52552.2


[I 2022-09-09 13:14:34,977] Trial 38 finished with value: 21152.946709786273 and parameters: {'reg_lambda': 2.7630799986965487e-05, 'max_bin': 392, 'num_leaves': 119}. Best is trial 0 with value: 19869.439942355075.
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', '定期借家', '建物構造', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters fr

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003812 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2454
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[1]	training's rmse: 70543.5	valid_1's rmse: 71246.1
Training until validation scores don't improve for 10 rounds
[2]	training's rmse: 67659.1	valid_1's rmse: 68698.9
[3]	training's rmse: 64926.5	valid_1's rmse: 66217.8
[4]	training's rmse: 62353.4	valid_1's rmse: 63954.1
[5]	training's rmse: 59908.9	valid_1's rmse: 61794.1
[6]	training's rmse: 57589.6	valid_1's rmse: 59728.5
[7]	training's rmse: 55409.5	valid_1's rmse: 57833.3
[8]	training's rmse: 53380.5	valid_1's rmse: 56098.3
[9]	training's rmse: 51429.7	valid_1's rmse: 54409.4
[10]	training's rmse: 49559.3	valid_1's rmse: 52830.5
[11]	training's rmse: 47830	valid_1's rmse: 51390.5
[12]	t

[I 2022-09-09 13:14:51,715] Trial 39 finished with value: 20640.18176104962 and parameters: {'reg_lambda': 5.543941231681784e-06, 'max_bin': 560, 'num_leaves': 104}. Best is trial 0 with value: 19869.439942355075.


{'reg_lambda': 4.430375245218262e-06, 'max_bin': 502, 'num_leaves': 97}

In [66]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D"]], buildings], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D"]],test_buildings], axis=1)

category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','建物構造']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'reg_lambda': 4.430375245218262e-06,
    'max_bin': 502,
    'num_leaves': 97
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
display(importance)


c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', '定期借家', '建物構造', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001315 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2338
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 35522.8	valid_1's rmse: 41119
[20]	training's rmse: 20999.8	valid_1's rmse: 30029.8
[30]	training's rmse: 15766.6	valid_1's rmse: 26169.6
[40]	training's rmse: 13470.8	valid_1's rmse: 24666.9
[50]	training's rmse: 12167.3	valid_1's rmse: 23997.3
[60]	training's rmse: 11211.1	valid_1's rmse: 23438.5
[70]	training's rmse: 10402.8	valid_1's rmse: 22959.8
[80]	training's rmse: 9724.98	valid_1's rmse: 22708.4
[90]	training's rmse: 9124.37	valid_1's rmse: 22473.9
[100]	training's rmse: 8603.84	valid_1's 

,importance
築年数,3.957611e+13
面積,2.419552e+14
契約期間,6.641633e+12
定期借家,1.841904e+12
所在階,4.469821e+12
全体の階数,3.555017e+13
最寄り駅,7.145723e+13
所要時間,8.618902e+12
所在地,2.742508e+13
部屋数,2.697620e+12


アンサンブルのために結果を出力する。

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib

###訓練データの読み込み
house_age = pd.read_csv('house_age.csv')
area_size = pd.read_csv('area_size.csv')
room_arrange = pd.read_csv('room_arrange.csv')
contract_span = pd.read_csv('contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('reg_rent.csv')
floor_scores = pd.read_csv('floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('stations.csv')
minits = pd.read_csv('minits.csv')
addresses = pd.read_csv('addresses.csv')
room_arrange = pd.read_csv('room_arrange.csv')
buildings = pd.read_csv('buildings.csv')

rent = pd.read_csv('rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('test_house_age.csv')
test_area_size = pd.read_csv('test_area_size.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_contract_span = pd.read_csv('test_contract_span.csv')
test_contract_span2 = pd.read_csv('test_contract_span2.csv')
test_reg_rent = pd.read_csv('test_reg_rent.csv')
test_floor_scores = pd.read_csv('test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('test_capital_floor_scores2.csv')
test_stations = pd.read_csv('test_stations.csv')
test_minits = pd.read_csv('test_minits.csv')
test_addresses = pd.read_csv('test_addresses.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_buildings = pd.read_csv('test_buildings.csv')

In [72]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K"]]], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D","K"]]], axis=1)

category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','K']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'reg_lambda': 4.430375245218262e-06,
    'max_bin': 502,
    'num_leaves': 97
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
display(importance)


c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future rel

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002869 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2338
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 35409.7	valid_1's rmse: 41017.7
[20]	training's rmse: 20992.8	valid_1's rmse: 29987.9
[30]	training's rmse: 15816.3	valid_1's rmse: 26143
[40]	training's rmse: 13448.1	valid_1's rmse: 24530.4
[50]	training's rmse: 12097.3	valid_1's rmse: 23854.2
[60]	training's rmse: 11187.3	valid_1's rmse: 23345.5
[70]	training's rmse: 10408.8	valid_1's rmse: 22980.3
[80]	training's rmse: 9712	valid_1's rmse: 22660.9
[90]	training's rmse: 9103.33	valid_1's rmse: 22366.5
[100]	training's rmse: 8602.63	valid_1's rmse: 22142.6
[110]	training's rmse: 8220.1	valid_1's rmse: 21988.4

,importance
築年数,4.031494e+13
面積,2.434538e+14
契約期間,6.287400e+12
定期借家,1.866982e+12
所在階,4.686737e+12
全体の階数,3.580639e+13
最寄り駅,7.062095e+13
所要時間,8.290599e+12
所在地,2.796149e+13
部屋数,2.570306e+12


In [73]:
y_pred = pd.DataFrame(y_pred)
y_pred.to_csv('LGBM_result.csv',index=False)

In [74]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K"]]], axis=1)
y_pred = model.predict(X_train, num_iteration=model.best_iteration)
y_pred = pd.DataFrame(y_pred)
y_pred.to_csv('en_LGBM_result.csv',index=False)